# Overview

Observability for all model types (LLM, NLP, CV, Tabular)

Phoenix Inferences allows you to observe the performance of your model through visualizing all the model’s inferences in one interactive UMAP view.

This powerful visualization can be leveraged during EDA to understand model drift, find low performing clusters, uncover retrieval issues, and export data for retraining / fine tuning.

# Quickstart

The following Quickstart can be executed in a Jupyter notebook or Google Colab.

We will begin by logging just a training set. Then proceed to add a production set for comparison.

## Step 1: Install and load dependencies

Use `pip` or `conda` to install `arize-phoenix`.

In [ ]:
%pip install -Uq 'arize-phoenix[embeddings]'

## Step 2: Prepare Model Data

Phoenix visualizes data taken from pandas dataframe, where each row of the dataframe compasses all the information about each inference (including feature values, prediction, metadata, etc.)

For this Quickstart, we will show an example of visualizing the inferences from a computer vision model. See example notebooks for all model types [here](https://docs.arize.com/phoenix/notebooks).

Let’s begin by working with the training set for this model.

### Download the dataset and load it into a Pandas dataframe.

In [ ]:
import pandas as pd

import phoenix as px

train_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/cv/human-actions/human_actions_training.parquet"
)

### Preview the dataframe (optional)

Note that each row contains all the data specific to this CV model for each inference.

In [ ]:
train_df.head()

## Step 3: Define a Schema

Before we can log these inferences, we need to define a Schema object to describe them.

The Schema object informs Phoenix of the fields that the columns of the dataframe should map to.

Here we define a Schema to describe our particular CV training set:

In [ ]:
train_schema = px.Schema(
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="predicted_action",
    actual_label_column_name="actual_action",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="image_vector",
            link_to_data_column_name="url",
        ),
    },
)

***Important:*** The fields used in a Schema will vary depending on the model type that you are working with.

For examples on how Schema are defined for other model types (NLP, tabular, LLM-based applications), see example notebooks under [https://docs.arize.com/phoenix/notebooks#embedding-analysis](Embedding Analysis) and [https://docs.arize.com/phoenix/notebooks#structured-data-analysis](Structured Data Analysis).

## Step 4: Wrap into Inference Object

Wrap your `train_df` and schema `train_schema` into a Phoenix `inferences` object:

In [ ]:
train_ds = px.Inferences(dataframe=train_df, schema=train_schema, name="training")

## Step 5: Launch Phoenix!

We are now ready to launch Phoenix with our Inferences!

Here, we are passing `train_ds` as the `primary` inferences, as we are only visualizing one inference set (see Step 6 for adding additional inference sets).

In [ ]:
session = px.launch_app(primary=train_ds)

Running this will fire up a Phoenix visualization. Follow in the instructions in the output to view Phoenix in a browser, or in-line in your notebook. 
Optional - try the following exercises to familiarize yourself more with Phoenix:


**You are now ready to observe the training set of your model!**

![Alt text](https://docs.arize.com/~gitbook/image?url=https%3A%2F%2F3394180728-files.gitbook.io%2F%7E%2Ffiles%2Fv0%2Fb%2Fgitbook-x-prod.appspot.com%2Fo%2Fspaces%252FShR775Rt7OzHRfy5j2Ks%252Fuploads%252Fws8mEYR4wy4LX4VPTKmg%252FScreenshot%25202023-11-06%2520at%252012.28.17%25E2%2580%25AFPM.png%3Falt%3Dmedia%26token%3D14a4ea3d-6982-4757-9fbd-7a9ab14374d0&width=768&dpr=4&quality=100&sign=20443e1&sv=1)

![Alt text](https://docs.arize.com/~gitbook/image?url=https%3A%2F%2Fstorage.googleapis.com%2Farize-assets%2Fphoenix%2Fassets%2Fimages%2FHDBSCAN_drift_analysis.png&width=768&dpr=4&quality=100&sign=18f42777&sv=1)

# Optional actions and activities

## Checkpoint A - Exercises to familiarize yourself more with Phoenix:

- [ ] Click on `image_embedding` under the Embeddings section to enter the UMAP projector view
- [ ] Select a point where the model accuracy is <0.78, and see the embedding visualization below update to include only points from this selected timeframe
- [ ] Select the cluster with the lowest accuracy; from the list of automatic clusters generated by Phoenix
  - Note that Phoenix automatically generates clusters for you on your data using a clustering algorithm called HDBSCAN (more information: [https://docs.arize.com/phoenix/concepts/embeddings-analysis#clusters](https://docs.arize.com/phoenix/concepts/embeddings-analysis#clusters)
- [ ] Change the colorization of your plot - e.g. select Color By ‘correctness’, and ‘dimension'
- [ ] Describe in words an insight you've gathered from this visualization

*Discuss your answers in our [https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email](community)!*

## Add comparison data

In order to visualize drift, conduct A/B model comparisons, or in the case of an information retrieval use case, compare inferences against a [corpus](https://docs.arize.com/phoenix/~/changes/v6Zhm276x8LlKmwqElIA/how-to/define-your-schema/corpus-data), you will need to add a comparison dataset to your visualization.

We will continue on with our CV model example above, and add a set of production data from our model to our visualization.

This will allow us to analyze drift and conduct A/B comparisons of our production data against our training set.

### Prepare production inferences

In [ ]:
prod_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/cv/human-actions/human_actions_training.parquet"
)

#### View Data (Optional)

In [ ]:
prod_df.head()

### Define model schema

Note that this schema differs slightly from our `train_schema` above, as our `prod_df` does not have a ground truth column!

In [ ]:
prod_schema = px.Schema(
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="predicted_action",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="image_vector",
            link_to_data_column_name="url",
        ),
    },
)

***When do I need a different schema?***

In general, if both sets of inferences you are visualizing have identical schemas, you can reuse the Schema object.

However, there are often differences between the schema of a primary and reference dataset. For example:
- Your production set does not include any ground truth, but your training set does.
- Your primary dataset is the set of prompt-responses in an LLM application, and your reference is your corpus.
- Your production data has differing timestamps between all inferences, but your training set does not have a timestamp column.

Read more about comparison dataset Schemas here: [How many schemas do I need?](https://docs.arize.com/phoenix/~/changes/v6Zhm276x8LlKmwqElIA/quickstart/phoenix-inferences/inferences)

### Wrap into Inferences object

In [ ]:
prod_ds = px.Inferences(dataframe=prod_df, schema=prod_schema, name="production")

### Launch Phoenix with both Inferences!

This time, we will include both `train_ds` and `prod_ds` when calling `launch_app`.

In [ ]:
session = px.launch_app(primary=prod_ds, reference=train_ds)

***What data should I set as `reference` and as `primary`?***

Select the inferences that you want to use as the referential baseline as your `reference`, and the dataset you'd like to actively evaluate as your `primary`.

In this case, training is our referential baseline, for which we want to gauge the behavior (e.g. evaluate drift) of our production data against.

Once again, enter your Phoenix app with the new link generated by your session.

You are now ready to conduct comparative Root Cause Analysis!

![Alt text](https://docs.arize.com/~gitbook/image?url=https%3A%2F%2F3394180728-files.gitbook.io%2F%7E%2Ffiles%2Fv0%2Fb%2Fgitbook-x-prod.appspot.com%2Fo%2Fspaces%252FShR775Rt7OzHRfy5j2Ks%252Fuploads%252FNrbA8YpM4n476tghpcay%252FScreenshot%25202023-11-06%2520at%25209.43.45%25E2%2580%25AFAM.png%3Falt%3Dmedia%26token%3D632c3273-7950-40e4-a708-58bfc34ebd7c&width=768&dpr=4&quality=100&sign=8eecdfa8&sv=1)

### Checkpoint B - try the following exercises to familiarize yourself more with Phoenix:

- [ ] Click into `image_embedding` under the Embeddings listing to enter the UMAP projector  
- [ ] Select a point on the time series where there is high drift (hint: as given by Euclidean Distance), and see the datapoints from the time selection being rendered below  
- [ ] While colorizing the data by 'Dataset', select the datapoints with the lasso tool where there exists only production data (hint: this is a set of data that has emerged in prod, and is a cause for the increase in drift!)  
- [ ] Export the selected cluster from Phoenix  
- [ ] Describe in words the process you went through to understand increased drift in your production data  

Discuss your answers in our [community](https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email)!

## Export data

Once you have identified datapoints of interest, you can export this data directly from the Phoenix app for further analysis, or to incorporate these into downstream model retraining and finetuning flows.

See more on exporting data [here](https://docs.arize.com/phoenix/inferences/how-to-inferences/export-your-data).

## Enable production observability with Arize

Once your model is ready for production, you can add Arize to enable production-grade observability. Phoenix works in conjunction with Arize to enable end-to-end model development and observability.

With Arize, you will additionally benefit from:
- Being able to publish and observe your models in real-time as inferences are being served, and/or via direct connectors from your table/storage solution
- Scalable compute to handle billions of predictions
- Ability to set up monitors & alerts
- Production-grade observability
- Integration with Phoenix for model iteration to observability
- Enterprise-grade RBAC and SSO
- Experiment with infinite permutations of model versions and filters

Create your [free account](https://arize.com/join) and see the full suite of [Arize](https://docs.arize.com/arize/) features.

# Where to go from here?

Read more about Embeddings Analysis [here](https://docs.arize.com/phoenix/inferences/use-cases-inferences/embeddings-analysis)

# Questions

Join the [Phoenix Slack community](https://join.slack.com/t/arize-ai/shared_invite/zt-1ppbtg5dd-1CYmQO4dWF4zvXFiONTjMg) to ask questions, share findings, provide feedback, and connect with other developers.